In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, GRU, Dense,Reshape

In [9]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-2/DRUG-AE.txt', delimiter='|', header=None)
df.columns = ['PubMed-ID', 'Sentence', 'Adverse-Effect', 'Begin_Offset_AE', 'End_Offset_AE', 'Drug', 'Begin_Offset_Drug', 'End_Offset_Drug']

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Sentence'])
vocab_size = len(tokenizer.word_index) + 1

In [11]:
max_len= max([len(seq) for seq in df['Sentence']])
max_len

568

In [12]:
# Pad sequences
sequences = tokenizer.texts_to_sequences(df['Sentence'])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

In [13]:
# Create target labels
labels = np.zeros((len(df), max_len))
for idx, row in df.iterrows():
    sentence = row['Sentence']
    word = row['Adverse-Effect']
    try:
        position = sentence.find(word)
        labels[idx, position:position+len(word)] = 1
    except ValueError:
        print(f"The word '{word}' is not found in the list.")
        print(sentence)
        print(idx)

In [14]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

In [15]:
# Define model architectures
def create_lstm_model():
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_len))
    model.add(LSTM(128, return_sequences=True))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def create_bilstm_model():
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_len))
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def create_gru_model():
    model = Sequential()
    model.add(Embedding(vocab_size, 100, input_length=max_len))
    model.add(GRU(128, return_sequences=True))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [16]:
# Train models
lstm_model = create_lstm_model()
lstm_model.summary()
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

bilstm_model = create_bilstm_model()
bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

gru_model = create_gru_model()
gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 568, 100)          724500    
                                                                 
 lstm (LSTM)                 (None, 568, 128)          117248    
                                                                 
 dense (Dense)               (None, 568, 1)            129       
                                                                 
Total params: 841877 (3.21 MB)
Trainable params: 841877 (3.21 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
137/137 [==============================] - 27s 146ms/step - loss: 0.1628 - accuracy: 0.9605 - val_loss: 0.1342 - val_accuracy: 0.9678
Epoch 2/10
137/137 [==============================] - 9s 63ms/step - loss: 0.1359 - accuracy: 0.9681 - val_loss: 0.1330 - val_accuracy: 0.9

In [17]:
# Evaluate models
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_binary = np.round(y_pred)
    accuracy = accuracy_score(y_test.flatten(), y_pred_binary.flatten())
    precision = precision_score(y_test.flatten(), y_pred_binary.flatten())
    recall = recall_score(y_test.flatten(), y_pred_binary.flatten())
    f1 = f1_score(y_test.flatten(), y_pred_binary.flatten())
    roc_auc = roc_auc_score(y_test.flatten(), y_pred.flatten())
    return accuracy, precision, recall, f1, roc_auc

In [18]:
lstm_results = evaluate_model(lstm_model, X_test, y_test)
bilstm_results = evaluate_model(bilstm_model, X_test, y_test)
gru_results = evaluate_model(gru_model, X_test, y_test)

print("LSTM Results:")
print("Accuracy:", lstm_results[0])
print("Precision:", lstm_results[1])
print("Recall:", lstm_results[2])
print("F1-score:", lstm_results[3])
print("ROC AUC:", lstm_results[4])

print("\nBiLSTM Results:")
print("Accuracy:", bilstm_results[0])
print("Precision:", bilstm_results[1])
print("Recall:", bilstm_results[2])
print("F1-score:", bilstm_results[3])
print("ROC AUC:", bilstm_results[4])

print("\nGRU Results:")
print("Accuracy:", gru_results[0])
print("Precision:", gru_results[1])
print("Recall:", gru_results[2])
print("F1-score:", gru_results[3])
print("ROC AUC:", gru_results[4])

43/43 [==============================] - 1s 12ms/step
LSTM Results:
Accuracy: 0.9678584326471651
Precision: 0.5572767259514247
Recall: 0.11418942748540303
F1-score: 0.18954078314036688
ROC AUC: 0.9113338357460573

BiLSTM Results:
Accuracy: 0.9670716607336326
Precision: 0.4978951396861845
Recall: 0.050981621536894076
F1-score: 0.09249253519124129
ROC AUC: 0.8576854107149032

GRU Results:
Accuracy: 0.9660914203167724
Precision: 0.46871195519844167
Recall: 0.22630197108037148
F1-score: 0.30523255813953487
ROC AUC: 0.9318455275383012
